In [123]:
#import the library sqlite3 
import sqlite3

db = "data/exemple.db"
conn = sqlite3.connect(db)

# # create a data cursor cur in order execute SQL statements and fetch results from SQL queries
cur = conn.cursor()

# # Create table with the column date, side, symbol, qty, price
create_table_1 = """ CREATE TABLE IF NOT EXISTS projects (
                                        date DATETIME CURRENT_TIME,
                                        side text NOT NULL,
                                        symbol text,
                                        qty int,
                                        price float
                                    ); """

cur.execute(create_table_1) 

# Insert 3 row of data: the stock APPLE ( AAPL) with the date of transaction, the nature of side ( BUY or SELL),
# the quantity of the stock traded and the price
insert_data = """INSERT INTO projects VALUES
                ('2021-01-01','BUY','APPL',100,148.48),
                ('2021-01-02','SELL','APPL',100,149.11),
                ('2021-01-03','BUY','APPL',100,146.39)
                """
cur.execute(insert_data) 
# Save (commit) the changes
cur.execute("""COMMIT""")

#return all resulting rows
cur.execute("SELECT * FROM projects")
data = cur.fetchall()
for row in data:
    print(row)

# Close the connection
conn.close()

('2021-01-01', 'BUY', 'APPL', 100, 148.48)
('2021-01-02', 'SELL', 'APPL', 100, 149.11)
('2021-01-03', 'BUY', 'APPL', 100, 146.39)


In [124]:
data

[('2021-01-01', 'BUY', 'APPL', 100, 148.48),
 ('2021-01-02', 'SELL', 'APPL', 100, 149.11),
 ('2021-01-03', 'BUY', 'APPL', 100, 146.39)]

## the construction of a financial database

Let’s start simple and only consider the structure for the equity data that we are gathering for now – in our case the constituents of the S&P500 index. We can tag on the various tables and possibly whole databases to the mix as and when we need to, i.e. when we start to deal with the spot currency data and fund data that we plan to include.

the  plan is to segregate theequity data into 4 separate tables, using unique ids and foreign keys to create the relevant relationships and links between them.

The tables will be as follows:

<img src='tables.png'>

In [134]:
#import the library os and the module Error of the library sqlite3
import os
import sqlite3
from sqlite3 import Error

def create_connection(db_file):
    
    """ create a database connection to the SQLite database
        specified by db_file
    :param db_file: database file
    :return: Connection object or None
    """
    conn = None
    try:
        conn = sqlite3.connect(db_file)
        print('connexion database ok')
    except Error as e:
        print(e)
    return conn



def create_table(conn, create_table_sql):
    """ create a table from the create_table_sql statement
    :param conn: Connection object
    :param create_table_sql: a CREATE TABLE statement
    :return:
    """
    try:
        c = conn.cursor()
        c.execute(create_table_sql)
    except Error as e:
        print(e)
        

#the name of the database
db_name = 'financial.db'      
database = r"{}/data/{}".format(os.getcwd(), db_name)
conn = create_connection(database)
conn.close()

connexion database ok


In [135]:
def main():    
        
#     # create the tables according to the MCD
    sql_create_exchange_table = """ CREATE TABLE IF NOT EXISTS exchange 
    ( `id` INTEGER PRIMARY KEY AUTOINCREMENT , 
      `name` varchar(100),
      `currency` varchar(20),
      `code` varchar(20)
     ) ;"""
    
    sql_create_company_table = """ CREATE TABLE IF NOT EXISTS company
    ( `id` INTEGER PRIMARY KEY AUTOINCREMENT , 
      `name` varchar(100),
      `industry` varchar(50),
      `sector` varchar(50),
      `hq_location` varchar(100),
      `security_id` INTEGER,
      FOREIGN KEY (`security_id`) REFERENCES security(id)
     ) """
    
    
    sql_create_security_table = """ CREATE TABLE IF NOT EXISTS security
    ( `id` INTEGER PRIMARY KEY AUTOINCREMENT , 
      `ticker` varchar(20),
      `name` varchar(50),
      `company_id` INTERGER,
      `exchange_id` INTEGER,
      FOREIGN KEY (`company_id`) REFERENCES company(id),
      FOREIGN KEY (`exchange_id`) REFERENCES exchange(id)
     ) ;"""
    
    sql_create_security_price_table =  """ CREATE TABLE IF NOT EXISTS security_price
    ( `id` INTEGER PRIMARY KEY AUTOINCREMENT , 
      `date` DATETIME,
      `open` float,
      `high` float,
      `low` float,
      `close` float,
      `volume` float,
      `adj_close` float,
      `security_id` INTEGER,
      FOREIGN KEY (`security_id`) REFERENCES security(id)
      ) """

    
    # create a database connection
    conn = create_connection(database)
    
    # create tables
    cur = conn.cursor()

    if conn is not None:
        cur.execute('''PRAGMA foreign_keys = ON;''')
        # create exchange table
        cur.execute(sql_create_exchange_table)         
        # create company table
        cur.execute(sql_create_company_table)             
        # create security table
        cur.execute(sql_create_security_table)            
        # create security_price table
        cur.execute(sql_create_security_price_table) 
    else:
        print("Error! cannot create the database connection.")
        
    conn.close()
main()

connexion database ok


In [136]:
#import the library pandas , requests and io
import pandas as pd
import requests
import io

In [174]:
exchange_data = pd.read_csv("data/ISO10383_MIC.csv", encoding='latin1')
exchange_data.head(10)

,MIC,OPERATING MIC,OPRT/SGMT,MARKET NAME-INSTITUTION DESCRIPTION,LEGAL ENTITY NAME,LEI,MARKET CATEGORY CODE,ACRONYM,ISO COUNTRY CODE (ISO 3166),CITY,WEBSITE,STATUS,CREATION DATE,LAST UPDATE DATE,LAST VALIDATION DATE,EXPIRY DATE,COMMENTS
0,DRSP,DRSP,OPRT,EURONEXT UK - REPORTING SERVICES,EURONEXT LONDON LIMITED,969500HMVSZ0TCV65D58,APPA,NaN,GB,LONDON,WWW.EURONEXT.COM,ACTIVE,20210927,20210927,20210927.0,NaN,APPROVED PUBLICATION ARRANGEMENT.
1,XCNQ,XCNQ,OPRT,CANADIAN SECURITIES EXCHANGE,"CNSX MARKETS, INC.",NaN,RMKT,CSE LISTED,CA,TORONTO,WWW.THECSE.COM,ACTIVE,20090427,20210927,20210927.0,NaN,FORMERLY KNOWN AS PURE (CSE OTHER LISTED) FORM...
2,PURE,XCNQ,SGMT,CANADIAN SECURITIES EXCHANGE - PURE,"CNSX MARKETS, INC.",NaN,NSPD,CSE-PURE,CA,TORONTO,WWW.THECSE.COM,ACTIVE,20061225,20210927,20210927.0,NaN,FORMERLY KNOWN AS PURE (CSE OTHER LISTED) FORM...
3,ZODM,ZODM,OPRT,ZODIA MARKETS,ZODIA MARKETS HOLDINGS LIMITED,5493006GZOHISU7JTC27,NSPD,NaN,GB,LONDON,WWW.ZODIA-MARKETS.COM,ACTIVE,20210927,20210927,20210927.0,NaN,CRYPTOASSET BROKERAGE AND EXCHANGE.
4,FNFX,BGCF,SGMT,FENICS FX ECN,"FENICS FX, LLC",549300I0NONVB9S5DZ56,NSPD,NaN,US,NEW YORK,WWW.FENICSFX.COM,ACTIVE,20210927,20210927,20210927.0,NaN,ELECTRONIC COMMUNICATIONS NETWORK.
5,NORX,NORX,OPRT,NASDAQ OSLO ASA,NASDAQ OSLO ASA,5493003IZZWOHHJY1L88,RMKT,NaN,NO,OSLO,WWW.NASDAQ.COM/SOLUTIONS/EUROPEAN-COMMODITIES,ACTIVE,20110523,20210927,20210927.0,NaN,NaN
6,POSE,POSE,OPRT,PORTFOLIO STOCK EXCHANGE,"EUROPEAN DIGITAL SECURITIES EXCHANGE, S.L.",959800UP9ANDBHTKJ408,MLTF,NaN,ES,MADRID,WWW.PORTFOLIO.EXCHANGE,ACTIVE,20210927,20210927,NaN,NaN,MULTILATERAL TRADING FACILITY.
7,PUND,PUND,OPRT,PUNDION LLC,PUNDION LLC,549300FOYST42PEQ1R51,NSPD,NaN,US,NEW YORK,WWW.PUNDION.COM,ACTIVE,20210927,20210927,20210927.0,NaN,EQUITY PRODUCTS.
8,UCBG,UCBG,OPRT,UNICREDIT BULBANK AD,UNICREDIT BULBANK AD,549300Z7V2WOFIMUEK50,SINT,NaN,BG,SOFIA,WWW.UNICREDITBULBANK.BG,ACTIVE,20210927,20210927,20210927.0,NaN,SYSTEMATIC INTERNALISER.
9,NZFX,XASX,SGMT,ASX - NEW ZEALAND FUTURES & OPTIONS,AUSTRALIAN SECURITIES EXCHANGE (ASX),549300SDXT36XYPVWI79,NSPD,ASX,AU,SYDNEY,WWW.ASX.COM.AU,ACTIVE,20070924,20210927,20210927.0,NaN,NaN


In [5]:
exchange_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2561 entries, 0 to 2560
Data columns (total 17 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   MIC                                  2561 non-null   object 
 1   OPERATING MIC                        2561 non-null   object 
 2   OPRT/SGMT                            2561 non-null   object 
 3   MARKET NAME-INSTITUTION DESCRIPTION  2561 non-null   object 
 4   LEGAL ENTITY NAME                    495 non-null    object 
 5   LEI                                  1833 non-null   object 
 6   MARKET CATEGORY CODE                 2559 non-null   object 
 7   ACRONYM                              890 non-null    object 
 8   ISO COUNTRY CODE (ISO 3166)          2560 non-null   object 
 9   CITY                                 2559 non-null   object 
 10  WEBSITE                              2387 non-null   object 
 11  STATUS                        

In [138]:
exchange_data = exchange_data[['ISO COUNTRY CODE (ISO 3166)', 'MIC', 'MARKET NAME-INSTITUTION DESCRIPTION', 'ACRONYM']]
exchange_data.head()


,ISO COUNTRY CODE (ISO 3166),MIC,MARKET NAME-INSTITUTION DESCRIPTION,ACRONYM
0,GB,DRSP,EURONEXT UK - REPORTING SERVICES,NaN
1,CA,XCNQ,CANADIAN SECURITIES EXCHANGE,CSE LISTED
2,CA,PURE,CANADIAN SECURITIES EXCHANGE - PURE,CSE-PURE
3,GB,ZODM,ZODIA MARKETS,NaN
4,US,FNFX,FENICS FX ECN,NaN


In [139]:
exchange_data.rename(columns={'ISO COUNTRY CODE (ISO 3166)' : 'country_code', 
                              'MIC':'code', 
                              'MARKET NAME-INSTITUTION DESCRIPTION': 'name',
                              'ACRONYM':'acronym'}, inplace=True)
exchange_data.head()

,country_code,code,name,acronym
0,GB,DRSP,EURONEXT UK - REPORTING SERVICES,NaN
1,CA,XCNQ,CANADIAN SECURITIES EXCHANGE,CSE LISTED
2,CA,PURE,CANADIAN SECURITIES EXCHANGE - PURE,CSE-PURE
3,GB,ZODM,ZODIA MARKETS,NaN
4,US,FNFX,FENICS FX ECN,NaN


In [140]:
exchange_data['id'] = exchange_data.index
exchange_data.head()

,country_code,code,name,acronym,id
0,GB,DRSP,EURONEXT UK - REPORTING SERVICES,NaN,0
1,CA,XCNQ,CANADIAN SECURITIES EXCHANGE,CSE LISTED,1
2,CA,PURE,CANADIAN SECURITIES EXCHANGE - PURE,CSE-PURE,2
3,GB,ZODM,ZODIA MARKETS,NaN,3
4,US,FNFX,FENICS FX ECN,NaN,4


In [148]:
# Get the most used currency
exchange_data.groupby(by="country_code").size().sort_values(ascending = False)

country_code
GB    566
US    420
DE    187
NL    104
FR     98
     ... 
GE      1
MN      1
MK      1
MG      1
KW      1
Length: 147, dtype: int64

In [149]:
# create the column currency in the the dataframe exhange_data whose values 
#are mapped with the country code : US => USD, GB =>GBP, DE =>EUR

currency_mapper={
    "GB":"GBP",
    "US":"USD",
    "DE":"EUR",
    "NL":"EUR",
    "FR":"EUR"
}
exchange_data['currency'] = exchange_data['country_code'].map(currency_mapper)
exchange_data[['id', 'name', 'currency', 'code']].head(30)


,id,name,currency,code
0,0,EURONEXT UK - REPORTING SERVICES,GBP,DRSP
1,1,CANADIAN SECURITIES EXCHANGE,NaN,XCNQ
2,2,CANADIAN SECURITIES EXCHANGE - PURE,NaN,PURE
3,3,ZODIA MARKETS,GBP,ZODM
4,4,FENICS FX ECN,USD,FNFX
5,5,NASDAQ OSLO ASA,NaN,NORX
6,6,PORTFOLIO STOCK EXCHANGE,NaN,POSE
7,7,PUNDION LLC,USD,PUND
8,8,UNICREDIT BULBANK AD,NaN,UCBG
9,9,ASX - NEW ZEALAND FUTURES & OPTIONS,NaN,NZFX


In [155]:
exchange_data = exchange_data[['id', 'name', 'currency', 'code']].head(30)

# get columns name
cols = "`,`".join([str(i) for i in exchange_data.columns.tolist()])

# create a database connection
conn = create_connection(database)

# insert in table
cur = conn.cursor()
for i,row in exchange_data.iterrows():
    sql = "INSERT INTO `exchange` (`" +cols + "`) VALUES (?,?,?,?)"
    cur.execute(sql, tuple(row))
    # the connection is not autocommitted by default, so we must commit to save our changes
    conn.commit()
    
conn.close()

connexion database ok


In [158]:
# scrape wiki table with symbols and details of Dow Jones constituents
dj_constituents = pd.read_html('https://en.wikipedia.org/wiki/Dow_Jones_Industrial_Average#Components')[1]
dj_constituents.rename(columns={'Company':'name', 'Industry': 'industry', 'Symbol': 'ticker'}, inplace=True)
dj_constituents.head(30)

,name,Exchange,ticker,industry,Date added,Notes,Index weighting
0,3M,NYSE,MMM,Conglomerate,1976-08-09,As Minnesota Mining and Manufacturing,2.88%
1,American Express,NYSE,AXP,Financial services,1982-08-30,NaN,3.56%
2,Amgen,NASDAQ,AMGN,Biopharmaceutical,2020-08-31,NaN,4.88%
3,Apple,NASDAQ,AAPL,Information technology,2015-03-19,NaN,3.15%
4,Boeing,NYSE,BA,Aerospace and defense,1987-03-12,NaN,3.40%
5,Caterpillar,NYSE,CAT,Construction and Mining,1991-05-06,NaN,4.19%
6,Chevron,NYSE,CVX,Petroleum industry,2008-02-19,Also 1930-07-18 to 1999-11-01,3.05%
7,Cisco,NASDAQ,CSCO,Information technology,2009-06-08,NaN,1.00%
8,Coca-Cola,NYSE,KO,Drink industry,1987-03-12,Also 1932-05-26 to 1935-11-20,1.28%
9,Disney,NYSE,DIS,Broadcasting and entertainment,1991-05-06,NaN,2.32%


In [159]:
# scrape wiki table with symbols and details of s&P500 constituents
sp_constituents = pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')[0]#.to_csv('constintuents_current.csv', index=False)
sp_constituents.rename(columns={'Symbol': 'ticker', 'Security':'name', 'Headquarters Location': 'hq_location',
                                'GICS Sector': 'sector', 'GICS Sub-Industry':'industry'},inplace=True)
sp_constituents.head()

,ticker,name,SEC filings,sector,industry,hq_location,Date first added,CIK,Founded
0,MMM,3M,reports,Industrials,Industrial Conglomerates,"Saint Paul, Minnesota",1976-08-09,66740,1902
1,AOS,A. O. Smith,reports,Industrials,Building Products,"Milwaukee, Wisconsin",2017-07-26,91142,1916
2,ABT,Abbott,reports,Health Care,Health Care Equipment,"North Chicago, Illinois",1964-03-31,1800,1888
3,ABBV,AbbVie,reports,Health Care,Pharmaceuticals,"North Chicago, Illinois",2012-12-31,1551152,2013 (1888)
4,ABMD,Abiomed,reports,Health Care,Health Care Equipment,"Danvers, Massachusetts",2018-05-31,815094,1981


In [160]:
#concetenate the previous tables/dataframe sp_constituents & dj_constituents with  the columns'name', 'industry', 'sector', 'hq_location'
company_table = pd.concat([dj_constituents, sp_constituents] )
company_table = company_table[['name','industry','sector','hq_location']]
#drop duplicate
initial_length = len(company_table)
company_table.drop_duplicates(keep='first', inplace=True, subset='name')
print("Il y avait {} doublons".format(initial_length - len(company_table)))
#sort
company_table = company_table.sort_values(by='name')
#reset index
company_table = company_table.reset_index()
company_table.drop('index', axis=1, inplace = True)
#create the colum id based on the index
company_table['id'] = company_table.index

Il y avait 19 doublons


In [161]:
company_table.head(25)

,name,industry,sector,hq_location,id
0,3M,Conglomerate,NaN,NaN,0
1,A. O. Smith,Building Products,Industrials,"Milwaukee, Wisconsin",1
2,ADM,Agricultural Products,Consumer Staples,"Chicago, Illinois",2
3,ADP,Data Processing & Outsourced Services,Information Technology,"Roseland, New Jersey",3
4,AES Corporation,Independent Power Producers & Energy Traders,Utilities,"Arlington, Virginia",4
5,AMD,Semiconductors,Information Technology,"Santa Clara, California",5
6,APA Corporation,Oil & Gas Exploration & Production,Energy,"Houston, Texas",6
7,AT&T,Integrated Telecommunication Services,Communication Services,"Dallas, Texas",7
8,AbbVie,Pharmaceuticals,Health Care,"North Chicago, Illinois",8
9,Abbott,Health Care Equipment,Health Care,"North Chicago, Illinois",9


In [162]:
#concetenate the previous tables/dataframe sp_constituents & dj_constituents with  the columns'ticker', 'name'
security_table = pd.concat([dj_constituents, sp_constituents] )
security_table = security_table[['ticker','name']]
#drop duplicate
initial_length = len(security_table)
security_table.drop_duplicates(keep='first', inplace=True, subset='name')
print("Il y avait {} doublons".format(initial_length - len(security_table)))
#sort
security_table = security_table.sort_values(by='ticker')
#reset index
security_table = security_table.reset_index()
security_table.drop('index', axis=1, inplace = True)
#create the colum id based on the index
security_table['id'] = security_table.index

Il y avait 19 doublons


In [163]:
security_table

,ticker,name,id
0,A,Agilent Technologies,0
1,AAL,American Airlines Group,1
2,AAP,Advance Auto Parts,2
3,AAPL,Apple Inc.,3
4,AAPL,Apple,4
...,...,...,...
509,YUM,Yum! Brands,509
510,ZBH,Zimmer Biomet,510
511,ZBRA,Zebra Technologies,511
512,ZION,Zions Bancorporation,512


In [164]:
security_table

,ticker,name,id
0,A,Agilent Technologies,0
1,AAL,American Airlines Group,1
2,AAP,Advance Auto Parts,2
3,AAPL,Apple Inc.,3
4,AAPL,Apple,4
...,...,...,...
509,YUM,Yum! Brands,509
510,ZBH,Zimmer Biomet,510
511,ZBRA,Zebra Technologies,511
512,ZION,Zions Bancorporation,512


In [165]:
#create the foreign key :  populating the security table with the company_id
#first step : create a dictionary with key, value pairs being the name of the company and the corresponding id 
#within its own table, which will be our foreign key in the second table.
company_id_mapper = dict( zip( company_table['name'],company_table['id'] ))
company_id_mapper

{'3M': 0,
 'A. O. Smith': 1,
 'ADM': 2,
 'ADP': 3,
 'AES Corporation': 4,
 'AMD': 5,
 'APA Corporation': 6,
 'AT&T': 7,
 'AbbVie': 8,
 'Abbott': 9,
 'Abiomed': 10,
 'Accenture': 11,
 'Activision Blizzard': 12,
 'Adobe Inc.': 13,
 'Advance Auto Parts': 14,
 'Aflac': 15,
 'Agilent Technologies': 16,
 'Air Products and Chemicals': 17,
 'Akamai': 18,
 'Alaska Air Group': 19,
 'Albemarle Corporation': 20,
 'Alexandria Real Estate Equities': 21,
 'Align Technology': 22,
 'Allegion': 23,
 'Alliant Energy': 24,
 'Allstate': 25,
 'Alphabet Inc. (Class A)': 26,
 'Alphabet Inc. (Class C)': 27,
 'Altria': 28,
 'Amazon': 29,
 'Amcor': 30,
 'Ameren': 31,
 'American Airlines Group': 32,
 'American Electric Power': 33,
 'American Express': 34,
 'American International Group': 35,
 'American Tower': 36,
 'American Water Works': 37,
 'Ameriprise Financial': 38,
 'AmerisourceBergen': 39,
 'Ametek': 40,
 'Amgen': 41,
 'Amphenol': 42,
 'Analog Devices': 43,
 'Ansys': 44,
 'Aon': 45,
 'Apple': 46,
 'Apple I

In [166]:
#Seconde step: create the foreign key company id 
#YOUR CODE
security_table['company_id'] = security_table['name'].map(company_id_mapper)

In [167]:
security_table

,ticker,name,id,company_id
0,A,Agilent Technologies,0,16
1,AAL,American Airlines Group,1,32
2,AAP,Advance Auto Parts,2,14
3,AAPL,Apple Inc.,3,47
4,AAPL,Apple,4,46
...,...,...,...,...
509,YUM,Yum! Brands,509,508
510,ZBH,Zimmer Biomet,510,510
511,ZBRA,Zebra Technologies,511,509
512,ZION,Zions Bancorporation,512,511


In [168]:
security_table

,ticker,name,id,company_id
0,A,Agilent Technologies,0,16
1,AAL,American Airlines Group,1,32
2,AAP,Advance Auto Parts,2,14
3,AAPL,Apple Inc.,3,47
4,AAPL,Apple,4,46
...,...,...,...,...
509,YUM,Yum! Brands,509,508
510,ZBH,Zimmer Biomet,510,510
511,ZBRA,Zebra Technologies,511,509
512,ZION,Zions Bancorporation,512,511


In [171]:
#create the foreign key : populate the company table with the security_id:
security_id_mapper = dict( zip( security_table['name'],security_table['id'] ))
company_table['security_id'] = company_table['name'].map(security_id_mapper)
company_table

,name,industry,sector,hq_location,id,security_id
0,3M,Conglomerate,NaN,NaN,0,313
1,A. O. Smith,Building Products,Industrials,"Milwaukee, Wisconsin",1,40
2,ADM,Agricultural Products,Consumer Staples,"Chicago, Illinois",2,13
3,ADP,Data Processing & Outsourced Services,Information Technology,"Roseland, New Jersey",3,14
4,AES Corporation,Independent Power Producers & Energy Traders,Utilities,"Arlington, Virginia",4,18
...,...,...,...,...,...,...
509,Zebra Technologies,Electronic Equipment & Instruments,Information Technology,"Lincolnshire, Illinois",509,511
510,Zimmer Biomet,Health Care Equipment,Health Care,"Warsaw, Indiana",510,510
511,Zions Bancorporation,Regional Banks,Financials,"Salt Lake City, Utah",511,512
512,Zoetis,Pharmaceuticals,Health Care,"Parsippany, New Jersey",512,513


In [175]:
#Reda the nyse.csv and the nasdaq.csv
#YOUR CODE 
nyse = pd.read_csv("data/nyse.csv")
nasdaq = pd.read_csv("data/nasdaq.csv")
nyse.head()

,Symbol,Name,LastSale,MarketCap,IPOyear,Sector,industry,Summary Quote,Unnamed: 8
0,DDD,3D Systems Corporation,7.66,$927.89M,NaN,Technology,Computer Software: Prepackaged Software,https://old.nasdaq.com/symbol/ddd,NaN
1,MMM,3M Company,169.80,$97.81B,NaN,Health Care,Medical/Dental Instruments,https://old.nasdaq.com/symbol/mmm,NaN
2,WBAI,500.com Limited,2.95,$126.85M,2013.0,Consumer Services,Services-Misc. Amusement & Recreation,https://old.nasdaq.com/symbol/wbai,NaN
3,EGHT,8x8 Inc,16.75,$1.75B,NaN,Technology,EDP Services,https://old.nasdaq.com/symbol/eght,NaN
4,AHC,A.H. Belo Corporation,1.44,$34.39M,NaN,Consumer Services,Newspapers/Magazines,https://old.nasdaq.com/symbol/ahc,NaN


In [176]:
nasdaq.sort_values(by="Symbol")

,Symbol,Name,LastSale,MarketCap,ADR TSO,IPOyear,Sector,Industry,Summary Quote,Unnamed: 9
336,AACG,ATA Creativity Global,1.230,1.456645e+07,11842642.0,NaN,Consumer Services,Other Consumer Services,https://old.nasdaq.com/symbol/aacg,NaN
312,AACQ,Artius Acquisition Inc.,9.710,8.793619e+08,NaN,2020.0,Finance,Business Services,https://old.nasdaq.com/symbol/aacq,NaN
313,AACQU,Artius Acquisition Inc.,10.090,0.000000e+00,NaN,2020.0,Finance,Business Services,https://old.nasdaq.com/symbol/aacqu,NaN
314,AACQW,Artius Acquisition Inc.,1.305,0.000000e+00,NaN,2020.0,Finance,Business Services,https://old.nasdaq.com/symbol/aacqw,NaN
201,AAL,"American Airlines Group, Inc.",12.600,6.407871e+09,NaN,NaN,Transportation,Air Freight/Delivery Services,https://old.nasdaq.com/symbol/aal,NaN
...,...,...,...,...,...,...,...,...,...,...
433,ZTEST,BATS BZX Exchange,10.100,0.000000e+00,NaN,NaN,NaN,NaN,https://old.nasdaq.com/symbol/ztest,NaN
3835,ZUMZ,Zumiez Inc.,31.190,7.936311e+08,NaN,2005.0,Consumer Services,Clothing/Shoe/Accessory Stores,https://old.nasdaq.com/symbol/zumz,NaN
3833,ZVO,Zovio Inc.,4.190,1.346618e+08,NaN,NaN,Consumer Services,Other Consumer Services,https://old.nasdaq.com/symbol/zvo,NaN
3837,ZYNE,"Zynerba Pharmaceuticals, Inc.",3.720,1.088293e+08,NaN,2015.0,Health Care,Major Pharmaceuticals,https://old.nasdaq.com/symbol/zyne,NaN


In [188]:
def assign_exchange_id(value) :
    if value in nasdaq.Symbol.values:
        return 1300
    if value in nyse.Symbol.values:
        return 367
    
#populate the exchange_id column : assign the correct exchange_id for each stock in the security table:
# the exchange_id of  will be 1300 for the nyse stocks and 367 for the nasdaq stocks
security_table['exchange_id'] = security_table.apply(lambda x : assign_exchange_id(x['ticker']), axis = 1)
security_table

,ticker,name,id,company_id,exchange_id
0,A,Agilent Technologies,0,16,367.0
1,AAL,American Airlines Group,1,32,1300.0
2,AAP,Advance Auto Parts,2,14,367.0
3,AAPL,Apple Inc.,3,47,1300.0
4,AAPL,Apple,4,46,1300.0
...,...,...,...,...,...
509,YUM,Yum! Brands,509,508,367.0
510,ZBH,Zimmer Biomet,510,510,367.0
511,ZBRA,Zebra Technologies,511,509,1300.0
512,ZION,Zions Bancorporation,512,511,1300.0


In [191]:
security_table.groupby('exchange_id').size()

exchange_id
367.0     359
1300.0    144
dtype: int64

In [199]:
#save the data in the sql table security and company of the database db_name

# get columns name
cols_security = "`,`".join([str(i) for i in security_table.columns.tolist()])
cols_company = "`,`".join([str(i) for i in company_table.columns.tolist()])

# create a database connection
conn = create_connection(database)

# insert in table security
cur = conn.cursor()
for i,row in security_table.iterrows():
    sql = "INSERT INTO `security` (`" +cols_security + "`) VALUES (?,?,?,?,?)"
    cur.execute(sql, tuple(row))
    conn.commit()
    
# insert in table security
cur = conn.cursor()
for i,row in company_table.iterrows():
    sql = "INSERT INTO `company` (`" +cols_company + "`) VALUES (?,?,?,?,?,?)"
    cur.execute(sql, tuple(row))
    conn.commit()    

    
conn.close()

connexion database ok


IntegrityError: UNIQUE constraint failed: company.id

In [115]:
#Extraction the prices of YAHOO: WARNING this action may take 15 min !!!
from tqdm import tqdm_notebook as tqdm
import pandas_datareader.data as web
stock_pricing_dfs = []
for stock_id in tqdm(security_table['id']):
    try:
        stock_pricing_df = web.DataReader(security_table.iloc[stock_id]['ticker'], 
                           start='2010-1-1', 
                           end='2020-08-31', 
                           data_source='yahoo')
        stock_pricing_df['security_id'] = stock_id
        stock_pricing_dfs.append(stock_pricing_df)
    except:
        pass
security_price_table = pd.concat(stock_pricing_dfs)

C:\Users\utilisateur\AppData\Local\Temp\ipykernel_2188\451006209.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for stock_id in tqdm(security_table['id']):


  0%|          | 0/514 [00:00<?, ?it/s]

In [202]:
security_price_table.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1104474 entries, 0 to 1104473
Data columns (total 9 columns):
 #   Column       Non-Null Count    Dtype         
---  ------       --------------    -----         
 0   Date         1104474 non-null  datetime64[ns]
 1   high         1104474 non-null  float64       
 2   low          1104474 non-null  float64       
 3   open         1104474 non-null  float64       
 4   close        1104474 non-null  float64       
 5   volume       1104474 non-null  float64       
 6   adj_close    1104474 non-null  float64       
 7   security_id  1104474 non-null  int64         
 8   id           1104474 non-null  int64         
dtypes: datetime64[ns](1), float64(6), int64(2)
memory usage: 75.8 MB


In [204]:
#insert data in the table security_price
security_price_table = security_price_table[['high', 'low', 'open', 'close', 'volume', 'adj_close', 'security_id']]
security_price_table.reset_index(inplace=True)
security_price_table['id'] = security_price_table.index
print(security_price_table.head())
security_price_table.drop('index', axis=1, inplace = True)

security_price_table.to_sql("security_price", conn, if_exists="append", index=False)

   index       high        low       open      close     volume  adj_close  \
0      0  22.625179  22.267525  22.453505  22.389128  3815561.0  20.331635   
1      1  22.331903  22.002861  22.324751  22.145924  4186031.0  20.110777   
2      2  22.174536  22.002861  22.067240  22.067240  3243779.0  20.039331   
3      3  22.045780  21.816881  22.017166  22.038628  3095172.0  20.013344   
4      4  22.067240  21.745352  21.917025  22.031473  3733918.0  20.006849   

   security_id  id  
0            0   0  
1            0   1  
2            0   2  
3            0   3  
4            0   4  


1104474